#   LangChain Built-In Tool (내장 도구)

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [33]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

---

## **내장 도구 (Built-in Tool)**

- **내장 도구**는 시스템에 **기본적으로 포함**된 사전 정의된 도구들의 집합

- 파일 처리, 웹 검색, 코드 실행 등 **자주 사용되는 기본 기능**들을 즉시 활용 가능

- 별도의 구현 없이 **바로 사용**할 수 있어 개발 시간과 노력을 절약 가능

---

### 1. **SQLDatabaseToolkit**

- **SQLDatabaseToolkit**은 LLM이 **데이터베이스와 상호작용**할 수 있게 해주는 특화 도구

- 자연어를 **SQL 쿼리로 변환**하고 데이터베이스 작업을 수행하는 기능 제공

- **테이블 스키마 분석**, **쿼리 실행**, **결과 해석** 등 DB 관련 작업을 자동화

`(1) SQLDatabaseToolkit 초기화`

In [4]:
# 데이터베이스 검색 (내장 도구)
from langchain_community.agent_toolkits import SQLDatabaseToolkit 
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI

# 데이터베이스 연결 및 도구 생성 (프로젝트2에서 생성한 ETF 데이터베이스 사용)
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
db = SQLDatabase.from_uri("sqlite:///etf_database.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# 도구 목록 출력
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000025C79772A80>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000025C79772A80>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000025C79772A80>),
 QuerySQLCheckerTool(description='Use this tool to 

`(2) QuerySQLDatabaseTool`

- **QuerySQLDatabaseTool**은 SQL 쿼리를 실행하고 데이터베이스 결과를 반환하는 도구
- 잘못된 쿼리 입력 시 **오류 메시지**를 통해 문제점 확인 가능
- **sql_db_schema** 도구와 연계하여 테이블 구조 확인 및 쿼리 수정 가능

In [5]:
from langchain_community.tools import QuerySQLDatabaseTool

# 데이터베이스 쿼리 도구 생성
query_excecuter = QuerySQLDatabaseTool(db=db)

# 데이터베이스 쿼리 실행
query = "SELECT count(*) FROM ETFs;"
result = query_excecuter.invoke(query)

# 결과 출력
print(result)

[(930,)]


### **[실습]**

- QuerySQLDatabaseTool 도구를 생성하고, ETFsInfo 테이블에 저장된 데이터 개수를 출력하세요.

In [6]:
# QuerySQLDatabaseTool 도구 생성 (이미 위에서 생성됨)
# query_excecuter = QuerySQLDatabaseTool(db=db)

# ETFsInfo 테이블의 데이터 개수 조회
query = "SELECT count(*) FROM ETFsInfo;"
result = query_excecuter.invoke(query)

# 결과 출력
print(f"ETFsInfo 테이블의 데이터 개수: {result}")

ETFsInfo 테이블의 데이터 개수: [(914,)]


`(3) ListSQLDatabaseTool`

- **ListSQLDatabaseTool**은 데이터베이스의 **전체 테이블 목록**을 조회하는 도구
- 다른 SQL 도구 사용 전 **테이블 존재 여부** 확인에 활용

In [7]:
from langchain_community.tools import ListSQLDatabaseTool

# 데이터베이스 리스트 도구 생성
list_excecuter = ListSQLDatabaseTool(db=db)

# 데이터베이스 리스트 실행
tables = list_excecuter.invoke("")

# 결과 출력
print(tables)

ETFs, ETFsInfo


`(4) InfoSQLDatabaseTool`

- **InfoSQLDatabaseTool**은 테이블의 **스키마 정보**와 **샘플 데이터**를 제공하는 도구
- **ListSQLDatabaseTool**을 먼저 실행하여 사용 가능한 테이블 목록 확인 필요
- 여러 테이블 정보를 한 번에 조회할 때는 **쉼표로 구분**하여 입력

In [8]:
from langchain_community.tools import InfoSQLDatabaseTool

# 데이터베이스 정보 도구 생성
info_excecuter = InfoSQLDatabaseTool(db=db)

# 데이터베이스 정보 실행
result = info_excecuter.invoke("ETFs")

# 결과 출력
print(result)


CREATE TABLE "ETFs" (
	"종목코드" TEXT, 
	"종목명" TEXT, 
	"상장일" TEXT, 
	"분류체계" TEXT, 
	"운용사" TEXT, 
	"수익률_최근1년" REAL, 
	"기초지수" TEXT, 
	"추적오차" REAL, 
	"순자산총액" REAL, 
	"괴리율" REAL, 
	"변동성" TEXT, 
	"복제방법" TEXT, 
	"총보수" REAL, 
	"과세유형" TEXT, 
	PRIMARY KEY ("종목코드")
)

/*
3 rows from ETFs table:
종목코드	종목명	상장일	분류체계	운용사	수익률_최근1년	기초지수	추적오차	순자산총액	괴리율	변동성	복제방법	총보수	과세유형
466400	1Q 25-08 회사채(A+이상)액티브	2023/09/19	채권-회사채-단기	하나자산운용	4.52	KIS 2025-08만기형 크레딧 A+이상 지수(총수익)	0.11	111916276404.0	0.03	매우낮음	실물(액티브)	0.1	배당소득세(보유기간과세)
491610	1Q CD금리액티브(합성)	2024/09/24	기타	하나자산운용	0.0	KIS 하나 CD금리 총수익지수	0.05	316206006696.0	0.02	매우낮음	합성(액티브)	0.02	배당소득세(보유기간과세)
451060	1Q K200액티브	2023/01/31	주식-시장대표	하나자산운용	-3.66	코스피 200	0.77	99754348820.0	-0.01	높음	실물(액티브)	0.18	배당소득세(보유기간과세)
*/


### **[실습]**

- InfoSQLDatabaseTool 도구를 생성하고, ETFsInfo 테이블의 스키마를 출력하세요.

In [9]:
# InfoSQLDatabaseTool 도구 생성 (이미 위에서 생성됨)
# info_excecuter = InfoSQLDatabaseTool(db=db)

# ETFsInfo 테이블의 스키마 정보 조회
result = info_excecuter.invoke("ETFsInfo")

# 결과 출력
print("=== ETFsInfo 테이블 스키마 정보 ===")
print(result)

=== ETFsInfo 테이블 스키마 정보 ===

CREATE TABLE "ETFsInfo" (
	"한글명" TEXT, 
	"영문명" TEXT, 
	"종목코드" TEXT, 
	"상장일" TEXT, 
	"펀드형태" TEXT, 
	"기초지수명" TEXT, 
	"추적배수" TEXT, 
	"자산운용사" TEXT, 
	"지정참가회사" TEXT, 
	"총보수" REAL, 
	"회계기간" TEXT, 
	"과세유형" TEXT, 
	"분배금지급일" TEXT, 
	"홈페이지" TEXT, 
	"기초시장" TEXT, 
	"기초자산" TEXT, 
	"기본정보" TEXT, 
	"투자유의사항" TEXT, 
	PRIMARY KEY ("종목코드")
)

/*
3 rows from ETFsInfo table:
한글명	영문명	종목코드	상장일	펀드형태	기초지수명	추적배수	자산운용사	지정참가회사	총보수	회계기간	과세유형	분배금지급일	홈페이지	기초시장	기초자산	기본정보	투자유의사항
PLUS 국채선물3년	PLUS 3-Year Korea Treasury Bond Futures	298340	2018-06-08	수익증권형	F-KTB 지수	일반 (1)	한화자산운용	미래에셋대우, 메리츠종합금융증권, IBK투자증권, 한화투자증권	0.1	매년 1월 1일부터 12월 31일(최초 회계기간은 투자신탁의 최초설정일부터 해당년도 12월 31일)	배당소득세(보유기간과세)	？회계기간 종료일(회계기간 종료일이 영업일이 아닌 경우 그 직전 영업일)	http://www.arirangetf.com/	(국내) (주식외) (-)	(채권) (국공채) / (-) (중기) / (-)	？ 이 투자신탁은 국내 채권 및 채권관련 파생상품을 주된 투자대상자산으로 하고, 수익증권 1좌당 순자산가치의 변동률을 한국거래소에서 산출하여 공표하는 추적대상지수인 국채선물지수(F	？ 파생상품투자위험: 파생상품(선물 등 투자)은 작은 증거금으로 거액의 결제가 가능한 지렛대효과(레버리지 효과)로 인하여 기초자산에 직접 투자하는 경우에 비하여 높은 위험에 노출됩


`(5) QuerySQLCheckerTool`

- **QuerySQLCheckerTool**은 SQL 쿼리의 **오류를 자동 검사**하는 도구
- 문법적/논리적 오류를 찾아내고 **수정된 쿼리**를 제공
- 쿼리 실행 전 **사전 검증**으로 오류 발생 가능성 최소화

In [10]:
from langchain_community.tools import QuerySQLCheckerTool

# 데이터베이스 쿼리 검사 도구 생성
query_checker = QuerySQLCheckerTool(llm=llm, db=db)

# 데이터베이스 쿼리 검사 실행
query = "SELECT * FROM ETFs count 3;"
result = query_checker.invoke(query)

# 결과 출력
print(result)


SELECT * FROM ETFs LIMIT 3;


`(6) SQL 에이전트 생성 및 실행`

> - **`create_agent`** 사용
> - `system_prompt` 매개변수 사용

In [24]:
from langchain_classic import hub  # ← 이렇게 변경

# Hub에서 프롬프트 템플릿 로드
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

# 시스템 메시지 생성
system_message = prompt_template.format(dialect="SQLite", top_k=5)

# 출력
print(system_message)


System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see w

In [25]:
from langchain.agents import create_agent

# SQL 에이전트 생성 
agent = create_agent(
    model=llm,
    tools=toolkit.get_tools(),
    system_prompt=system_message
)

# 입력 스키마 출력 (선택사항)
agent.get_input_jsonschema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAn `AIMessage` is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model and standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'},
         {'additionalProperties': True, 'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'additionalProperties': True,
     'title': 'Additional Kwargs',
     'type': 'object'},
    'response_metadata': {'additionalProperties': True,
     'title': 'Response Metadata',
     'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default'

In [26]:
# 프롬프트 입력변수 확인
prompt_template.input_variables


['dialect', 'top_k']

In [27]:
# 에이전트 실행
response = agent.invoke(
    {"messages": [{"role": "user", "content": "ETF 종목은 모두 몇 개인가요?"}]},
)

# 에이전트 실행 결과 출력
pprint(response['messages'])

[HumanMessage(content='ETF 종목은 모두 몇 개인가요?', additional_kwargs={}, response_metadata={}, id='ab377145-f519-4988-abfb-7647961e5dab'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 553, 'total_tokens': 565, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_c064fdde7c', 'id': 'chatcmpl-CT7BDeAFa6Zyc215jj0KdEmMZYWrG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--62bdf281-ea1a-4dec-9806-2881af98f03f-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_xFXmFvnO9iuxtWl3u4S9TN6P', 'type': 'tool_call'}], usage_metadata={'input_tokens': 553, 'output_tokens': 12, 'total_tokens': 565, 'input_token_deta

In [28]:
for msg in response['messages']:
    msg.pretty_print()

================================ Human Message =================================

ETF 종목은 모두 몇 개인가요?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_xFXmFvnO9iuxtWl3u4S9TN6P)
 Call ID: call_xFXmFvnO9iuxtWl3u4S9TN6P
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

ETFs, ETFsInfo
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_B9XxFM9Dr0gnwIrwbjQDDGaC)
 Call ID: call_B9XxFM9Dr0gnwIrwbjQDDGaC
  Args:
    table_names: ETFs
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "ETFs" (
	"종목코드" TEXT, 
	"종목명" TEXT, 
	"상장일" TEXT, 
	"분류체계" TEXT, 
	"운용사" TEXT, 
	"수익률_최근1년" REAL, 
	"기초지수" TEXT, 
	"추적오차" REAL, 
	"순자산총액" REAL, 
	"괴리율" REAL, 
	"변동성" TEXT, 
	"복제방법" TEXT, 
	"총보수" REAL, 
	"과세유형" TEXT, 
	PRIMARY KEY ("종목코드")
)

/*
3 rows fro

---
### **[실습]**

- 시스템 프롬프트 내용을 확인하고, 개선할 포인트를 찾아서 적용합니다. 
- 개선된 프롬프트를 적용해서 SQL 쿼리 에이전트를 정의합니다. 
- 데이터베이스 내용을 쿼리하는 질문을 테스트합니다. 

In [29]:
# 개선된 시스템 프롬프트 작성
improved_system_message = """당신은 SQLite 데이터베이스 전문가입니다.

다음 규칙을 반드시 따르세요:
1. 쿼리 실행 전에 반드시 sql_db_list_tables로 테이블 목록을 확인하세요.
2. 테이블 스키마를 확인하기 위해 sql_db_schema 도구를 사용하세요.
3. 쿼리를 실행하기 전에 sql_db_query_checker로 검증하세요.
4. 최대 {top_k}개의 결과만 반환하세요.
5. 쿼리 결과는 명확하고 간결하게 설명하세요.
6. 한국어로 답변하세요.

사용 가능한 도구:
- sql_db_list_tables: 데이터베이스의 모든 테이블 목록 조회
- sql_db_schema: 특정 테이블의 스키마 정보 확인
- sql_db_query: SQL 쿼리 실행
- sql_db_query_checker: SQL 쿼리 문법 검증

데이터베이스 방언: {dialect}
"""

# 시스템 메시지 생성
formatted_prompt = improved_system_message.format(dialect="SQLite", top_k=5)

# 개선된 SQL 에이전트 생성
from langchain.agents import create_agent

improved_agent = create_agent(
    model=llm,
    tools=toolkit.get_tools(),
    system_prompt=formatted_prompt
)

print("개선된 SQL 에이전트가 생성되었습니다.\n")

# 테스트 쿼리 실행
test_query = "ETFs 테이블에서 운용사별 ETF 개수를 상위 5개만 알려주세요."
print(f"질문: {test_query}\n")

response = improved_agent.invoke(
    {"messages": [{"role": "user", "content": test_query}]}
)

# 최종 답변 출력
final_message = response['messages'][-1]
print("=== 최종 답변 ===")
print(final_message.content)

개선된 SQL 에이전트가 생성되었습니다.

질문: ETFs 테이블에서 운용사별 ETF 개수를 상위 5개만 알려주세요.

=== 최종 답변 ===
ETFs 테이블에서 운용사별 ETF 개수를 집계한 결과, 상위 5개 운용사는 다음과 같습니다:
1. 삼성자산운용 - 202개
2. 미래에셋자산운용 - 200개
3. 케이비자산운용 - 118개
4. 한국투자신탁운용 - 88개
5. 한화자산운용 - 65개

이상입니다.


---

### 2. **웹 검색(Tavily Search API)**

- **Tavily**는 AI 에이전트(LLM)를 위해 특별히 설계된 검색 엔진

- 검색 결과로 **제목, URL, 컨텐츠, 답변** 등 상세 정보 제공

- 개발자는 **월 1,000회** 무료 검색 쿼터 사용 가능

- **설치**: `uv add langchain-tavily` 또는 `pip install langchain-tavily`

- **인증키**: `TAVILY_API_KEY`

`(1) 도구 정의`

In [34]:
from langchain_tavily import TavilySearch

# Tavily 검색 도구 생성
tavily_search = TavilySearch(
    max_results=5, 
    topic="general",
)


`(2) 도구 직접 호출`
- 자연어 쿼리를 도구에 전달

In [35]:
result = tavily_search.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
pprint(result)

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?',
 'request_id': '03a5257b-1be3-4413-abe5-a0be328257a9',
 'response_time': 1.22,
 'results': [{'content': '본문 바로가기 # 국내 ETF 상품만 870개, 전세계 4위…앞서나가는 한국 ETF 시장 '
                         '다양한 상품 공급으로 금융 소비자 선택지 넓어져   870개 종목이 상장돼있다. 2002년 '
                         "10월 14일 'KODEX 200' 등 4개 종목(순자산총액 3552억원)으로 시작한 지 약 "
                         '23년만이다. 국내 ETF 시장은 전세계 시장과 비교해봐도 성장세가 가파르다. 같은 시점 '
                         '한국의 ETF 순자산 규모는 1105억달러(152조4513억원)이고 종목 수는 857개로 각각 '
                         "전세계에서 0.86%, 7.94% 비중을 차지했다. 2002년 시장 초기엔 'KODEX "
                         "200' 등 주식시장 전반의 움직임을 추종하는 상품밖에 없었지만 당장 올해 상장한 80개의 "
                         'ETF들만 보더라도 종류가 다양해졌다. 시장 성장 속도도 더욱 빨라지고 있다. 세계적 투자자 '
                         '워런 버핏이 이끄는 벅셔해서웨이(B주)와 이 회사가 투자하는 상위 10개 종목에 투자한다. '
                         '**관련 뉴스** \'SOL 미국AI 전력인프라 ETF\' 상장…"관련 밸류체인 총망라" '
                         

`(3) ToolCall을 사용한 호출`
- 모델에서 생성된 `ToolCall`을 사용하여 도구를 호출

In [36]:
from langchain_openai import ChatOpenAI

# 모델 생성
model = ChatOpenAI(model="gpt-4.1-mini")

# 모델에 도구 등록
model_with_tools = model.bind_tools([tavily_search])

# 사용자 쿼리를 입력하여 ToolCall 생성
response = model_with_tools.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")

# ToolCall 출력 
pprint(response.tool_calls)

[{'args': {'query': '한국 시장 ETF 종목 수'},
  'id': 'call_GEFwFlubYQpHV0zJFfX9vsCr',
  'name': 'tavily_search',
  'type': 'tool_call'}]


In [37]:
# ToolCall을 사용하여 도구 실행
model_generated_tool_call = response.tool_calls[0]
tool_msg = tavily_search.invoke(model_generated_tool_call)

# 도구 실행 결과 출력
pprint(tool_msg.content)

('{"query": "한국 시장 ETF 종목 수", "follow_up_questions": null, "answer": null, '
 '"images": [], "results": [{"url": '
 '"https://www.hankyung.com/article/202407142443i", "title": "국내 ETF 상품만 870개, '
 '전세계 4위…앞서나가는 한국 ETF 시장", "content": "본문 바로가기 # 국내 ETF 상품만 870개, 전세계 4위…앞서나가는 '
 "한국 ETF 시장 다양한 상품 공급으로 금융 소비자 선택지 넓어져   870개 종목이 상장돼있다. 2002년 10월 14일 'KODEX "
 "200' 등 4개 종목(순자산총액 3552억원)으로 시작한 지 약 23년만이다. 국내 ETF 시장은 전세계 시장과 비교해봐도 성장세가 "
 '가파르다. 같은 시점 한국의 ETF 순자산 규모는 1105억달러(152조4513억원)이고 종목 수는 857개로 각각 전세계에서 '
 "0.86%, 7.94% 비중을 차지했다. 2002년 시장 초기엔 'KODEX 200' 등 주식시장 전반의 움직임을 추종하는 상품밖에 "
 '없었지만 당장 올해 상장한 80개의 ETF들만 보더라도 종류가 다양해졌다. 시장 성장 속도도 더욱 빨라지고 있다. 세계적 투자자 워런 '
 "버핏이 이끄는 벅셔해서웨이(B주)와 이 회사가 투자하는 상위 10개 종목에 투자한다. **관련 뉴스** 'SOL 미국AI 전력인프라 "
 'ETF\' 상장…\\"관련 밸류체인 총망라\\" 기관들은 그동안 공매도 외에 현금담보부거래, 재대여거래 등 각기 다른 목적으로 빌린 '
 '주식을 포괄해 관리해왔다.... 상단 바로가기", "score": 0.92507464, "raw_content": null}, '
 '{"url": "https://www.kbsec.com/go.able?linkcd=m04010014", "title": "투자정보 > '
 '리서치보고서 > 포트폴리오 > 한국상

In [38]:
json.loads(tool_msg.content)['results']

[{'url': 'https://www.hankyung.com/article/202407142443i',
  'title': '국내 ETF 상품만 870개, 전세계 4위…앞서나가는 한국 ETF 시장',
  'content': '본문 바로가기 # 국내 ETF 상품만 870개, 전세계 4위…앞서나가는 한국 ETF 시장 다양한 상품 공급으로 금융 소비자 선택지 넓어져   870개 종목이 상장돼있다. 2002년 10월 14일 \'KODEX 200\' 등 4개 종목(순자산총액 3552억원)으로 시작한 지 약 23년만이다. 국내 ETF 시장은 전세계 시장과 비교해봐도 성장세가 가파르다. 같은 시점 한국의 ETF 순자산 규모는 1105억달러(152조4513억원)이고 종목 수는 857개로 각각 전세계에서 0.86%, 7.94% 비중을 차지했다. 2002년 시장 초기엔 \'KODEX 200\' 등 주식시장 전반의 움직임을 추종하는 상품밖에 없었지만 당장 올해 상장한 80개의 ETF들만 보더라도 종류가 다양해졌다. 시장 성장 속도도 더욱 빨라지고 있다. 세계적 투자자 워런 버핏이 이끄는 벅셔해서웨이(B주)와 이 회사가 투자하는 상위 10개 종목에 투자한다. **관련 뉴스** \'SOL 미국AI 전력인프라 ETF\' 상장…"관련 밸류체인 총망라" 기관들은 그동안 공매도 외에 현금담보부거래, 재대여거래 등 각기 다른 목적으로 빌린 주식을 포괄해 관리해왔다.... 상단 바로가기',
  'score': 0.92507464,
  'raw_content': None},
 {'url': 'https://www.kbsec.com/go.able?linkcd=m04010014',
  'title': '투자정보 > 리서치보고서 > 포트폴리오 > 한국상장ETF 추천종목',
  'content': '+ 한국상장ETF 추천종목 ## 한국상장ETF 추천종목 **한국 증시에 상장된 국내 및 해외 투자 ETF 중 중장기 모멘텀을 고려하여 ETF를 추천합니다.** :   한국 ETF 추천은 전체 추천 종목 수

`(4) LLM 체인과 연계`
- Tavily 도구를 언어 모델에 바인딩하고 사용자 입력을 처리하는 체인을 생성

In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import ToolMessage
import datetime

# 언어 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 프롬프트 템플릿 정의
today = datetime.datetime.today().strftime("%Y-%m-%d")
prompt = ChatPromptTemplate(
    [
        ("system", f"당신은 도움이 되는 어시스턴트입니다. 오늘 날짜는 {today}입니다."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# Tavily 도구를 모델에 바인딩
llm_with_tools = llm.bind_tools([tavily_search])

# 체인 생성
llm_chain = prompt | llm_with_tools

@chain  # 함수를 체인으로 사용하기 위해 데코레이터 추가
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    
    # 도구 호출 결과를 처리
    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        tool_msg = tavily_search.invoke(tool_call)

        search_results = json.loads(tool_msg.content)['results']

        tool_msg_parsed = ToolMessage(
            content=str(search_results),
            tool_call_id=tool_msg.tool_call_id  # tool_call_id 사용
        )
        tool_msgs.append(tool_msg_parsed)

    # 최종 결과 반환
    final_response = llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

    return final_response

# 체인 호출
response = tool_chain.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
print("===== 최종 응답 =====")
print(response.content)

===== 최종 응답 =====
한국 시장에서 거래되는 ETF 종목은 약 870개 정도입니다. 2002년 10월 14일 4개 종목으로 시작한 이후 약 23년 만에 870개 종목이 상장되어 있습니다. 전체 ETF 시장 규모도 크게 성장하였고, 다양한 상품이 공급되고 있습니다.


---

# [실습] **Wikipedia** 연습문제

#### **환경 설정**

- `WikipediaQueryRun` 도구를 사용하여 Wikipedia에서 정보를 검색하고 활용
- 아래 링크를 참조하여 필요한 환경설정을 처리 
- 참조: https://python.langchain.com/docs/integrations/tools/wikipedia/

#### **문제 1: 기본 정보 검색**
1. "Artificial Intelligence"라는 주제에 대해 Wikipedia에서 요약 정보를 검색하세요.
2. 검색 결과를 출력하세요.


In [41]:
# Wikipedia 도구 import 및 설정
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Wikipedia API 래퍼 생성 (한국어 지원, 최대 문서 로드 수 1개, 요약 문장 수 3개)
wikipedia = WikipediaAPIWrapper(
    lang="en",  # 영어로 검색
    top_k_results=1,  # 최대 1개 문서
    doc_content_chars_max=2000  # 최대 2000자
)

# WikipediaQueryRun 도구 생성
wiki_tool = WikipediaQueryRun(api_wrapper=wikipedia)

# "Artificial Intelligence" 검색
query = "Artificial Intelligence"
result = wiki_tool.invoke(query)

# 결과 출력
print(f"=== {query} Wikipedia 검색 결과 ===\n")
print(result)

=== Artificial Intelligence Wikipedia 검색 결과 ===

Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A 

#### **문제 2: 도구로 변환하여 Tool Calling으로 처리**
1. @tool 데코레이터를 사용하여 WikipediaQueryRun 객체를 Tool로 변환하세요.
1. "Quantum Computing"이라는 주제에 대해 Wikipedia에서 상세 정보를 검색하세요.
1. LLM 모델에 바인딩하고 도구를 호출해서 실행하세요. 

In [42]:
# @tool 데코레이터를 사용하여 Wikipedia 도구로 변환
from langchain_core.tools import tool

@tool
def search_wikipedia(query: str) -> str:
    """Wikipedia에서 정보를 검색합니다."""
    wiki_wrapper = WikipediaAPIWrapper(
        lang="en",
        top_k_results=1,
        doc_content_chars_max=2000
    )
    wiki_run = WikipediaQueryRun(api_wrapper=wiki_wrapper)
    return wiki_run.invoke(query)

# LLM 모델 생성 및 도구 바인딩
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
model_with_wiki = model.bind_tools([search_wikipedia])

# "Quantum Computing" 검색을 위한 사용자 쿼리
user_query = "Tell me about Quantum Computing"
response = model_with_wiki.invoke(user_query)

# ToolCall 확인
print("=== ToolCall 정보 ===")
pprint(response.tool_calls)

# ToolCall을 사용하여 도구 실행
if response.tool_calls:
    tool_call = response.tool_calls[0]
    tool_result = search_wikipedia.invoke(tool_call)
    
    print("\n=== Wikipedia 검색 결과 ===")
    print(tool_result.content[:500] + "...")  # 처음 500자만 출력
else:
    print("도구 호출이 없습니다.")

=== ToolCall 정보 ===
[{'args': {'query': 'Quantum Computing'},
  'id': 'call_1MkzqheZarDrISqU1dHhiOSV',
  'name': 'search_wikipedia',
  'type': 'tool_call'}]

=== Wikipedia 검색 결과 ===
Page: Quantum computing
Summary: A quantum computer is a (real or theoretical) computer that uses quantum mechanical phenomena in an essential way: it exploits superposed and entangled states, and the intrinsically non-deterministic outcomes of quantum measurements, as features of its computation. Quantum computers can be viewed as sampling from quantum systems that evolve in ways classically described as operating on an enormous number of possibilities simultaneously, though still subject to st...


#### **문제 3: 에이전트를 사용하여 도구 호출 및 실행**
1. LangChain AgentExecutor 또는 LangGraph Agent를 사용하세요. 
1. "Deep Learning은 무엇인가?" 라는 질문에 대해 Wikipedia 검색에 기반한 답변을 생성하세요. 
1. 에이전트의 실행과정과 최종 답변을 출력하세요. 

In [43]:
# LangGraph Agent를 사용한 Wikipedia 검색
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate

# Wikipedia 검색 도구 (이미 위에서 정의한 search_wikipedia 사용)

# 시스템 프롬프트 생성
system_prompt = """당신은 Wikipedia 정보를 검색하여 정확한 답변을 제공하는 AI 어시스턴트입니다.

다음 규칙을 따르세요:
1. 사용자 질문에 대한 정보를 Wikipedia에서 검색하세요.
2. 검색 결과를 바탕으로 명확하고 간결한 답변을 제공하세요.
3. 한국어로 답변하세요.
"""

# 프롬프트 템플릿 생성
wiki_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("placeholder", "{messages}"),
])

# 에이전트 생성
wiki_agent = create_react_agent(
    model=llm,  # 위에서 생성한 llm 사용
    tools=[search_wikipedia],
    prompt=wiki_prompt
)

print("Wikipedia 검색 에이전트가 생성되었습니다.\n")

# 에이전트 실행
question = "Deep Learning은 무엇인가?"
print(f"질문: {question}\n")

response = wiki_agent.invoke(
    {"messages": [{"role": "user", "content": question}]}
)

# 실행 과정 출력
print("=== 에이전트 실행 과정 ===")
for msg in response['messages']:
    msg.pretty_print()

# 최종 답변 출력
print("\n" + "="*100)
print("\n=== 최종 답변 ===")
final_message = response['messages'][-1]
print(final_message.content)

Wikipedia 검색 에이전트가 생성되었습니다.

질문: Deep Learning은 무엇인가?



<frozen abc>:106: LangGraphDeprecatedSinceV10: AgentStatePydantic has been moved to `langchain.agents`. Please update your import to `from langchain.agents import AgentStatePydantic`. Deprecated in LangGraph V1.0 to be removed in V2.0.
C:\Users\kaydash\AppData\Local\Temp\ipykernel_17160\2070091947.py:23: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  wiki_agent = create_react_agent(


=== 에이전트 실행 과정 ===
================================ Human Message =================================

Deep Learning은 무엇인가?
================================== Ai Message ==================================
Tool Calls:
  search_wikipedia (call_XSH24QzUCz4fDwpvqQnRJ7CB)
 Call ID: call_XSH24QzUCz4fDwpvqQnRJ7CB
  Args:
    query: Deep Learning
================================= Tool Message =================================
Name: search_wikipedia

Page: Deep learning
Summary: In machine learning, deep learning focuses on utilizing multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be supervised, semi-supervised or unsupervised.
Some common